In [8]:
import os
import numpy as np

from nltk.tokenize import word_tokenize
from natsort import natsorted
from nltk.stem import PorterStemmer
#preprocessing
       
files_name=natsorted(os.listdir('files'))
print('                                              *****    Documents     ***** ' )
document_of_terms=[]
for files in files_name:
    with open(f'files/{files}','r') as f:
        document=f.read()
        
        print(document)
 
    tokenized_documents=word_tokenize(document)
    terms=[]
    for word in tokenized_documents:
            terms.append(word)
    document_of_terms.append(terms)

print('                                               *****    Documents in  Terms   *****                                                                        \n    '
    ,  document_of_terms,'\n')                        

######### 
#positional_index

document_number = 0
positional_index = {}

print() 
print("                                               *****     positions    *****                                       \n   ")
for document in document_of_terms:
    for positional, term in enumerate(document):
        if term in positional_index:
            positional_index[term][0] = positional_index[term][0] + 1

            if document_number in positional_index[term][1]:
                positional_index[term][1][document_number].append(positional)
            else:
                positional_index[term][1][document_number] = [positional]

        else:
            positional_index[term] = []
            positional_index[term].append(1)
            positional_index[term].append({})
            positional_index[term][1][document_number] = [positional]

    document_number += 1
    

print(positional_index,'\n')

#query_preprocessing

query='fools fear'

final_list=[[]for i in range (10)]
print('                                    ***** all response list *****                                                     ')
for word in query.split():    
    if word in positional_index.keys():
        for key in positional_index[word][1].keys():
            #print(key)
            if final_list[key]!=[]:
                
                 if final_list[key][-1] == positional_index[word][1][key][0]-1:
                        final_list[key].append(positional_index[word][1][key][0])
        
            else:
                 final_list[key].append(positional_index[word][1][key][0])
            
print(final_list,'\n')

print('                                  *****response list to query*****                                              ')
for position,list in enumerate(final_list,start=1):
    if len(list)==len(query.split()):
      print(position,list)     


#find tf and wtf
import pandas as pd
import math


all_words=[]
for doc in document_of_terms:
    for word in doc:
        all_words.append(word)
 #tf       
def get_term_freq(doc):
    words_found=dict.fromkeys(all_words,0)
    for word in doc:
        words_found[word]+=1
    return words_found

term_freq=pd.DataFrame(get_term_freq(document_of_terms[0]).values(),index=get_term_freq(document_of_terms[0]).keys())              
print('                                  *****term freq for each  term in doc*****                                              ')
print()        
for i in range(1,len(document_of_terms)):
    term_freq[i]=get_term_freq(document_of_terms[i]).values()
    
term_freq.columns=['doc'+str(i) for i in range (1,11)]    
print(term_freq)     

#wtf

def get_weighted_term_freq(x):
    if x>0:
      return math.log(x)+1
    return 0
     
print('                                  ***** weighted term freq for each  term in doc*****                                              ')
print()        
for i in range(1,len(document_of_terms)+1):
    term_freq['doc'+str(i)]= term_freq['doc'+str(i)].apply(get_weighted_term_freq) 
    
print(term_freq,'\n')
print()
 
#idf and tf*idf
 
tfd=pd.DataFrame(columns=['df','idf'])

print('                                  *****        IDF          *****                                              ')
for i in range(len(term_freq)):
    
    frequancy=term_freq.iloc[i].values.sum()
    
    tfd.loc[i,'df']=frequancy
    tfd.loc[i,'idf']=math.log10(10/(float(frequancy)))
    
tfd.index=term_freq.index
    
print(tfd)
# tf*idf

tf_idf=term_freq.multiply(tfd['idf'],axis=0)

print('                                  *****         TF*IDF             *****                                              ')
print()      
print(tf_idf)
# doc length

'''
def get_documents_length(col):
    return np.sqrt(tf_idf[col].apply(lambda x:x**2).sum())


for column in tf_idf.columns:
    document_length.loc[0,column+'_len']=get_documents_length(column)
print('                                  *****         Doc Length             *****                                              ') 
print(document_length,'\n')    


# normalized tf*idf
norm_tf_idf = pd.DataFrame()
def get_normalized(col, x):
    try:
        return x / document_length[col + '__len'].values[0]
    except:
        return 0
for column in tf_idf.columns:
    norm_tf_idf[column] = tf_idf[column].apply(lambda x: get_normalized(column, x))

print('                                  *****        Normalized TF*IDF           *****                                              ')
print(norm_tf_idf, '\n')
'''

document_length=pd.DataFrame({
    f'doc{i}_length':np.sqrt(tf_idf[f'doc{i}'].apply(lambda x:x**2).sum()) for i in range (1,11)
                             
},index=[0])
print('                                  *****         Doc Length             *****                                              ') 
print()
print(document_length,'\n') 

norm_tf_idf= tf_idf.divide(document_length.values[0],axis=1)
print('                                  *****        Normalized TF*IDF           *****                                              ')
print()
print(norm_tf_idf)


                                              *****    Documents     ***** 
antony brutus caeser cleopatra mercy worser


antony brutus caeser calpurnia 
mercy worser
brutus caeser mercy worser
caeser mercy worser
antony caeser mercy 
angels fools fear in rush to tread where
angels fools fear in rush to tread where
angels fools in rush to tread where
fools fear in rush to tread where
                                               *****    Documents in  Terms   *****                                                                        
     [['antony', 'brutus', 'caeser', 'cleopatra', 'mercy', 'worser'], ['antony', 'brutus', 'caeser', 'calpurnia'], ['mercy', 'worser'], ['brutus', 'caeser', 'mercy', 'worser'], ['caeser', 'mercy', 'worser'], ['antony', 'caeser', 'mercy'], ['angels', 'fools', 'fear', 'in', 'rush', 'to', 'tread', 'where'], ['angels', 'fools', 'fear', 'in', 'rush', 'to', 'tread', 'where'], ['angels', 'fools', 'in', 'rush', 'to', 'tread', 'where'], ['fools', 'fear', 'in', 'rush', 'to', 'tread', 'where']] 


                                               *****     positions    *****            

In [9]:
q='antony brutus'

In [10]:
def get_w_tf(x):
    try:
        return math.log10(x)+1
    except:
        return 0

In [11]:
import math
query=pd.DataFrame(index=norm_tf_idf.index)
query['tf']=[1 if x in q.split() else 0 for x in list(norm_tf_idf.index)]
query['w_tf']=query['tf'].apply(lambda x:get_w_tf(x))
product=norm_tf_idf.multiply(query['w_tf'],axis=0)
query['idf']=tfd['idf']*query['w_tf']
query['tf_idf']=query['w_tf']*query['idf']
query['norm']=0
for i in range(len(query)):
    query['norm'].iloc[i]=float(query['idf'].iloc[i])/math.sqrt(sum(query['idf'].values**2))

product2=product.multiply(query['norm'],axis=0)   
#product2
query

TypeError: 'list' object is not callable

In [ ]:
math.sqrt(sum([x**2 for x in query['idf'].loc[q.split()]]))

0.7394622130520805

In [ ]:
query['idf'].loc[q.split()]

antony    0.522879
brutus    0.522879
Name: idf, dtype: object

In [ ]:
product2.loc[q.split()].values

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
scores={}
for col in product2.columns:
    if 0 in product2[col].loc[q.split()].values:
        pass
    else:
        scores[col]=product2[col].sum()

In [ ]:
scores

{}

In [ ]:
product2[list(scores.keys())]

""
antony
brutus
caeser
cleopatra
mercy
worser
calpurnia
angels
fools
fear


In [ ]:
product2[list(scores.keys())].loc[q.split()]

""
antony
brutus


In [ ]:
prod_res=product2[list(scores.keys())]

In [ ]:
prod_res.sum()

Series([], dtype: float64)

In [ ]:
final_score=sorted(scores.items(),key=lambda x:x[1],reverse=True)

In [ ]:
for doc in final_score:
    print(doc[0],end='')